## ニューラルネットワークモデル比較
## (sklearn vs pytorch vs 手動)
---

### モデル構造

- 入力層：特徴量数に依存（今回のデータセットでは8次元）  
- 隠れ層：16ユニット、活性化関数はシグモイド関数  
- 出力層：1ユニット、シグモイド活性化により確率を出力   

---



入力 X に対し、隠れ層・出力層の出力は以下のように計算：

$$
Z^{(1)} = X W_1 + b_1
$$

$$
A^{(1)} = \sigma(Z^{(1)})
$$

$$
Z^{(2)} = A^{(1)} W_2 + b_2
$$

$$
A^{(2)} = \sigma(Z^{(2)})
$$
 は入力特徴量の次元数

ニューロンの出力はシグモイド関数で計算：

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$



---

### 損失関数

出力 $A^{(2)}$ と正解ラベル $y$ ,サンプル数$m$を用いて損失は

$$
L = -\frac{1}{m} \sum_{i=1}^{m} \left[ y_i \log A_i^{(2)} + (1 - y_i) \log (1 - A_i^{(2)}) \right]
$$



---


### パラメータ更新（勾配降下法）

学習率 $\alpha$ を用いてパラメータを更新：

$$
W_1 \leftarrow W_1 - \alpha \nabla_{W_1}
$$

$$
b_1 \leftarrow b_1 - \alpha \nabla_{b_1}
$$

$$
W_2 \leftarrow W_2 - \alpha \nabla_{W_2}
$$

$$
b_2 \leftarrow b_2 - \alpha \nabla_{b_2}
$$

---



In [2]:
#ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
import time
import torch
import torch.nn as nn
import torch.optim as optim


In [ ]:
df = pd.read_csv("../datasets/diabetes.csv")
print(df.info())
X = df.drop("Outcome", axis=1).values
y = df["Outcome"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(y_train.shape)
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

#手動ニューラルネットワーク（1層）実装
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))

# 入力 → 中間層(16) → 出力(1)
m, input_dim = X_train_norm.shape
hidden_dim = 16
output_dim = 1
alpha = 0.1
epochs = 1000

#標準正規分布（平均0, 標準偏差1）で生成
W1 = np.random.randn(input_dim, hidden_dim)
b1 = np.zeros(hidden_dim)
W2 = np.random.randn(hidden_dim, output_dim)
b2 = np.zeros(output_dim)

start_time = time.time()
for epoch in range(epochs):
    # 計算
    Z1 = np.dot(X_train_norm, W1) + b1 #(m,hidden_dim)
    A1 = sigmoid(Z1) #(m,hidden_dim)
    Z2 = np.dot(A1, W2) + b2 #(m,output_dim)
    A2 = sigmoid(Z2)#(m,output_dim)
    
    # 損失（あえてaxis指定せずにスカラー化する）
    epsilon = 1e-7
    loss = -np.mean(y_train * np.log(A2 + epsilon) + (1 - y_train) * np.log(1 - A2 + epsilon))
    
    # 誤差逆伝搬法
    #L/Z2, L/W2 = L/Z2 * Z2/W2, L/A1 = L/Z2 * Z2/A1, L/Z1 = L/A1 * A1/Z1
    
    dZ2 = A2 - y_train #(m,output_dim)np.array([[0.1], [-0.2], [0.3], ×m])
    dW2 = np.dot(A1.T, dZ2) / m #(hidden_dim, output_dim) = (hidden_dim, m) ,dz([[0.1], [-0.2], [0.3], ...]) 
    db2 = np.mean(dZ2, axis=0) #(outout_dim,)
    dA1 = np.dot(dZ2, W2.T) #(m, hidden_dim)
    dZ1 = dA1 * sigmoid_derivative(Z1) #(m, hidden_dim)
    dW1 = np.dot(X_train_norm.T, dZ1) / m #(input_dim, hidden_dim) W ij一個ずつの勾配を、ミニバッチ内のサンプル平均で計算
    db1 = np.mean(dZ1, axis=0) #(hidden_dim,)
    
    # バッチ勾配降下法
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2


    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss = {loss:.4f}")

end_time = time.time()
print(f"time: {end_time - start_time:.4f} 秒")

# 評価
Z1_test = np.dot(X_test_norm, W1) + b1
A1_test = sigmoid(Z1_test)
Z2_test = np.dot(A1_test, W2) + b2
A2_test = sigmoid(Z2_test).flatten()
y_pred_manual = (A2_test >= 0.5).astype(int) #np.where(A2_test >= 0.5, 1, 0)
acc_manual = accuracy_score(y_test, y_pred_manual)
print(f"Accuracy: {acc_manual:.4f}")

""" 
#ミニバッチ
batch_size = 32
for epoch in range(epochs):
    for i in range(0, m, batch_size):
        X_batch = X_train_norm[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]

"""


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB
None
(614,)
Epoch 0: Loss = 0.9182
Epoch 100: Loss = 0.5544
Epoch 200: Loss = 0.4977
Epoch 300: Loss = 0.4780
Epoch 400: Loss = 0.4689
Epoch 500: Loss = 0.4638
Epoch 600: Loss = 0.4604
Epoch 700: Loss = 0.4579
Epoch 800: Loss = 0.4559
Epoch

' \n#ミニバッチ\nbatch_size = 32\nfor epoch in range(epochs):\n    for i in range(0, m, batch_size):\n        X_batch = X_train_norm[i:i+batch_size]\n        y_batch = y_train[i:i+batch_size]\n\n'

In [ ]:
# PyTorchによる実装
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__() #親クラス（nn.Module）の初期化処理を呼び出す 
        self.net = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.Sigmoid(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):#入力を受け取って出力を返す処理（順伝播）を定義
        return self.net(x)

# 入力次元
input_dim = X_train_norm.shape[1]
epochs = 1000


torch.manual_seed(42)
model = SimpleNN(input_dim) #モデル構造をメモリ上に作る（クラス → インスタンス）
criterion = nn.BCELoss() # クライテリオン=loss
optimizer = optim.SGD(model.parameters(), lr=0.1)

# テンソル(多次元配列)変換
X_train_tensor = torch.tensor(X_train_norm, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)

# 学習
start_time = time.time()
for epoch in range(epochs):
    y_pred = model(X_train_tensor) #= model.forward(X_train_tensor)  
    loss = criterion(y_pred, y_train_tensor)
    optimizer.zero_grad() #前のループで計算した勾配リセット
    loss.backward() #損失に対して逆伝播を実行し、各パラメータの勾配を計算
    optimizer.step() #計算した勾配を使って、モデルのパラメータを更新
    if epoch % 100 == 0:
        print(f"[PyTorch NN] Epoch {epoch}: Loss = {loss.item():.4f}")
end_time = time.time()
print(f"[PyTorch NN] 実行時間: {end_time - start_time:.4f} 秒")

# 評価
X_test_tensor = torch.tensor(X_test_norm, dtype=torch.float32)
y_pred_torch = model(X_test_tensor).detach().numpy().flatten() #detach() で計算グラフから切り離す
y_pred_torch_bin = (y_pred_torch >= 0.5).astype(int)
acc_torch = accuracy_score(y_test, y_pred_torch_bin)
print(f"Accuracy (PyTorch NN): {acc_torch:.4f}")


[PyTorch NN] Epoch 0: Loss = 0.7865
[PyTorch NN] Epoch 100: Loss = 0.5866
[PyTorch NN] Epoch 200: Loss = 0.5327
[PyTorch NN] Epoch 300: Loss = 0.4975
[PyTorch NN] Epoch 400: Loss = 0.4791
[PyTorch NN] Epoch 500: Loss = 0.4698
[PyTorch NN] Epoch 600: Loss = 0.4650
[PyTorch NN] Epoch 700: Loss = 0.4623
[PyTorch NN] Epoch 800: Loss = 0.4607
[PyTorch NN] Epoch 900: Loss = 0.4596
[PyTorch NN] 実行時間: 0.8992 秒
Accuracy (PyTorch NN): 0.7532


In [10]:
# sklearnによる実装
# モデル定義
model = MLPClassifier(hidden_layer_sizes=(16,),
                      activation='logistic',  # sigmoid活性化
                      solver='sgd',
                      learning_rate_init=0.1,
                      learning_rate = 'constant',
                      max_iter=1000,
                      random_state=42,
                      batch_size=X_train_norm.shape[0], 
                      verbose=True)  # 学習ログを表示

start_time = time.time()
model.fit(X_train_norm, y_train)
end_time = time.time()
# 予測
y_pred = model.predict(X_test_norm)
#　精度
acc = accuracy_score(y_test, y_pred)

print(f"Test accuracy: {acc:.4f}")
print(f"Training time: {end_time - start_time:.4f} seconds")

c:\Users\bsksh\Documents\programm\github\machine_learning\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.67893666
Iteration 2, loss = 0.67330685
Iteration 3, loss = 0.66765829
Iteration 4, loss = 0.66298742
Iteration 5, loss = 0.65940345
Iteration 6, loss = 0.65646312
Iteration 7, loss = 0.65358626
Iteration 8, loss = 0.65034892
Iteration 9, loss = 0.64659622
Iteration 10, loss = 0.64241006
Iteration 11, loss = 0.63800049
Iteration 12, loss = 0.63358660
Iteration 13, loss = 0.62931285
Iteration 14, loss = 0.62522003
Iteration 15, loss = 0.62126439
Iteration 16, loss = 0.61736233
Iteration 17, loss = 0.61343454
Iteration 18, loss = 0.60943248
Iteration 19, loss = 0.60534271
Iteration 20, loss = 0.60117583
Iteration 21, loss = 0.59695032
Iteration 22, loss = 0.59268006
Iteration 23, loss = 0.58836930
Iteration 24, loss = 0.58401442
Iteration 25, loss = 0.57960922
Iteration 26, loss = 0.57515047
Iteration 27, loss = 0.57064137
Iteration 28, loss = 0.56609229
Iteration 29, loss = 0.56151951
Iteration 30, loss = 0.55694289
Iteration 31, loss = 0.55238369
Iteration 32, los